# Cogent Labsの「Tegaki」APIに接続する
https://www.tegaki.ai/<br>
<br>
【はじめに】<br>
「1-1. ライブラリインポート、定数の読み込み」を実行します。<br>
「1-2. 関数の読み込み」を実行します。<br>
<br>
【手書き文字を認識させる】<br>
「2-1. 手書き文字認識用パラメータの読み込み」において、下記パラメータを適切なものに書き換えます。<br>
> テンプレートJSONファイル<br>
> 手書き画像が格納されているディレクトリ<br>
> 手書き画像_リクエストIDの紐づけファイルを出力するディレクトリ<br>

「2-2. 手書き文字認識リクエスト、手書き画像_リクエストID紐付け情報の出力」を実行します。<br>
<br>
【認識結果の取得】<br>
「3-1. 認識結果取得用パラメータの読み込み」において、下記パラメータを適切なものに書き換えます。<br>
> 手書き画像_リクエストIDの紐づけファイルを取得するディレクトリ<br>
> 認識結果を出力するディレクトリ<br>

「3-2. 認識結果取得」を実行します。<br>
<br>
【修正結果の取得】<br>
「4-1. 修正結果取得用パラメータの読み込み」において、下記パラメータを適切なものに書き換えます。<br>
> 手書き画像_リクエストIDの紐づけファイルを取得するディレクトリ<br>
> 修正結果を出力するディレクトリ<br>

「4-2. 認識結果取得」を実行します。<br>
<br>
【データの削除】<br>
「5-1. 削除対象リクエストID取得用パラメータの読み込み」において、下記パラメータを適切なものに書き換えます。<br>
> 手書き画像_リクエストIDの紐づけファイルを取得するディレクトリ<br>

「5-2. データ削除」を実行します。Response [204]が表示されれば、正常に削除されています。<br>



# ディレクトリ構成
教材ごとに複数のフォームを持つため、フォームごとにテンプレートを準備します。<br>
<img src="resources/0000_ポートフォリオ_日々の記録_テンプレート.png" style="width:800px"><br>
<br>
手書き画像をまとめて1つのディレクトリに配置します。<br>
<img src="resources/0001_ポートフォリオ_日々の記録_手書き画像.png" style="width:800px"><br>
<br>
認識結果にはリクエスト番号がそれぞれ採番されます。<br>
<img src="resources/0002_ポートフォリオ_日々の記録_認識結果.png" style="width:800px"><br>
<br>
認識結果と同じ番号で修正結果が上書きされます。<br>
<img src="resources/0003_ポートフォリオ_日々の記録_修正結果.png" style="width:800px"><br>
<br>
手書き画像とリクエスト番号の紐付けは対応表を参照します。<br>
<img src="resources/0004_ポートフォリオ_日々の記録_対応表.png" style="width:800px"><br>



# はじめに

In [1]:
# ========= 1-1. ライブラリインポート、定数の読み込み ==========

#!/usr/bin/python3

# ライブラリのインポート
import base64
import json
import requests
from IPython.display import display
#import fileupload
import glob
import pandas as pd
from datetime import datetime

# 定数
# APIエンドポイント（フォーム送信）
TEGAKI_FORM_ENDPOINT = 'https://api.tegaki.ai/hwr/v2/form'
# APIエンドポイント（リクエスト送信）
TEGAKI_REQUEST_ENDPOINT = 'https://api.tegaki.ai/hwr/v2/request/'

# アクセスキー
MY_API_KEY = '2b8b291f-164e-459d-80a6-d7bc93722cf6'


In [2]:
# ========= 1-2. 関数の読み込み ==========

"""
画像をbase64に変換する
"""
def encode_image(image):
    image_content = image.read()
    return base64.b64encode(image_content).decode('utf-8')


"""
Tegaki-APIに文字認識をリクエストする
リクエストIDが返却される
"""
def post_form(template_json_data, form_image_file):
    # Read json file
    #print(template_json_data)
    
    # Inject the base64-encoded form image into the template json
    template_json_data['imageData'] = encode_image(form_image_file)
    # Send POST request to Tegaki service
    response = requests.post(TEGAKI_FORM_ENDPOINT,
                             headers={'Authorization': 'apikey ' + MY_API_KEY},
                             json=template_json_data)
    # Print the result
    #print(response.json())
    return response.json()


"""
リクエストIDを元に、文字認識結果および修正結果をリクエストする
"""
def get_result(request_id, path_recognized_results):
    # Build the endpoint with the request id
    endpoint = TEGAKI_REQUEST_ENDPOINT + request_id

    # Send GET request
    response = requests.get(endpoint, headers={'Authorization': 'apikey {}'.format(MY_API_KEY)})
    
    path_output = path_recognized_results + "/" + request_id + ".json"
    with open(path_output, mode='w') as f:
        json.dump(response.json(), f, indent=4, ensure_ascii=False)

        
"""
テンプレートJSON、Base64画像を元に文字認識をリクエストする
"""
def get_request_id(path_template, path_image):
    # テンプレートのパス
    with open(path_template) as f:
        json_temp = json.load(f)
        
    # 画像の読み込み
    image = open(path_image, 'rb')
    response = post_form(json_temp, image)
    
    request_id = response["requestId"]
    return request_id


"""
手書き文字認識を実施する
手書き画像とリクエストIDの紐づけ情報をファイル出力する
"""
def exec_image_recognition(path_template, path_images_dir, path_images_reqid_dir):
    images_list = get_file_list(path_images_dir)

    image_reqid_list = []
    for path_img in images_list:
        image_reqid = [path_img, get_request_id(path_template, path_img)]
        image_reqid_list.append(image_reqid)
        
    exec_time = datetime.now().strftime("%Y%m%d%H%M%S")
    path_image_reqid_list = path_images_reqid_dir + "/" + str(exec_time) + ".tsv"
    df = pd.DataFrame(image_reqid_list)

    df.to_csv(path_image_reqid_list, sep="\t", header=False, index=False)
    

"""
リクエストIDを元に認識結果および修正結果をダウンロードする
"""
def exec_results_download(path_images_reqid_file, path_recognized_results):
    df_img_req = pd.read_csv(path_images_reqid_file, header=None, sep="\t")
    for index, row in df_img_req.iterrows():
        reqid = row[1]
        get_result(reqid, path_recognized_results_dir)
    
    
"""
ディレクトリ配下のファイル一覧を取得する
"""
def get_file_list(path_dir):
    files = glob.glob(path_dir + "/*")
    file_list = [fname for fname in files]
    return file_list


"""
リクエストIDを元にデータを削除する
"""
def exec_delete_data(path_images_reqid_file):
    df_img_req = pd.read_csv(path_images_reqid_file, header=None, sep="\t")
    for index, row in df_img_req.iterrows():
        reqid = row[1]
        delete_result(reqid)

        
# Get results from an already send request using the id
def delete_result(request_id):
    # Build the endpoint with the request id
    endpoint = TEGAKI_REQUEST_ENDPOINT + request_id
    
    # Send DELETE request
    response = requests.delete(endpoint, headers={'Authorization': 'apikey {}'.format(MY_API_KEY)})
    print(response)
    

# 手書き文字を認識させる

In [15]:
# ========= 2-1. 手書き文字認識用パラメータの読み込み ==========
# テンプレートJSONファイル
#path_template = "api_test_data/0001.ポートフォリオ/002.日々の記録/0000.テンプレート/hibi-form-jpg.json"
path_template = "api_test_data/0001.ポートフォリオ/100.実証第1校/0000.テンプレート/02.氏名なし/template-png_non_name.json"

# 手書き画像が格納されているディレクトリ
#path_images_dir = "api_test_data/0001.ポートフォリオ/002.日々の記録/0001.手書き画像"
path_images_dir = "api_test_data/0001.ポートフォリオ/100.実証第1校/0001.手書き画像"

# 手書き画像_リクエストIDの紐づけファイルを出力するディレクトリ
#path_images_reqid_dir = "api_test_data/0001.ポートフォリオ/002.日々の記録/0004.手書き画像_リクエストID_対応表"
path_images_reqid_dir = "api_test_data/0001.ポートフォリオ/100.実証第1校/0004.手書き画像_リクエストID_対応表"


In [16]:
# ========= 2-2. 手書き文字認識リクエスト、手書き画像_リクエストID紐付け情報の出力 ==========
exec_image_recognition(path_template, path_images_dir, path_images_reqid_dir)


# 認識結果の取得

In [17]:
# ========= 3-1. 認識結果取得用パラメータの読み込み ==========
# 手書き画像_リクエストIDの紐づけファイルを取得するディレクトリ
#path_images_reqid_file = "api_test_data/0001.ポートフォリオ/002.日々の記録/0004.手書き画像_リクエストID_対応表/20181219141832.tsv"
path_images_reqid_file = "api_test_data/0001.ポートフォリオ/100.実証第1校/0004.手書き画像_リクエストID_対応表/20181219145244.tsv"

# 認識結果を出力するディレクトリ
#path_recognized_results_dir = "api_test_data/0001.ポートフォリオ/002.日々の記録/0002.認識結果"
path_recognized_results_dir = "api_test_data/0001.ポートフォリオ/100.実証第1校/0002.認識結果"


In [18]:
# ========= 3-2. 認識結果取得 ==========
exec_results_download(path_images_reqid_file, path_recognized_results_dir)


# 修正結果の取得

In [21]:
# ========= 4-1. 修正結果取得用パラメータの読み込み ==========
# 手書き画像_リクエストIDの紐づけファイルを取得するディレクトリ
#path_images_reqid_file = "api_test_data/0001.ポートフォリオ/002.日々の記録/0004.手書き画像_リクエストID_対応表/20181219141832.tsv"
path_images_reqid_file = "api_test_data/0001.ポートフォリオ/100.実証第1校/0004.手書き画像_リクエストID_対応表/20181219145244.tsv"

# 修正結果を出力するディレクトリ
#path_recognized_results_dir = "api_test_data/0001.ポートフォリオ/002.日々の記録/0003.修正結果"
path_recognized_results_dir = "api_test_data/0001.ポートフォリオ/100.実証第1校/0003.修正結果"


In [22]:
# ========= 4-2. 修正結果取得 ==========
exec_results_download(path_images_reqid_file, path_recognized_results_dir)


# データの削除

In [23]:
# ========= 5-1. 削除対象リクエストID取得用パラメータの読み込み ==========
# 手書き画像_リクエストIDの紐づけファイルを取得するディレクトリ
#path_images_reqid_file = "api_test_data/0001.ポートフォリオ/002.日々の記録/0004.手書き画像_リクエストID_対応表/20181219141832.tsv"
path_images_reqid_file = "api_test_data/0001.ポートフォリオ/100.実証第1校/0004.手書き画像_リクエストID_対応表/20181219145244.tsv"


In [24]:
# ========= 5-2. データ削除 ==========
exec_delete_data(path_images_reqid_file)


<Response [204]>
<Response [204]>
